# Reminder Flow Hardening
The Gmail HITL agent now routes reminder actions through a durable dispatcher. Cancel and create operations are batched together so the SQLite store never observes a cancelled reminder without the corresponding recreation. The triage router only stages create actions immediately when it can continue straight to the response agent; notify flows defer creation until the human reviewer approves the reminder, keeping HITL semantics intact.

## Updated Reminder Graph
The mermaid diagram below highlights the dispatcher (`apply_reminder_actions_node`) and the deferred reminder queue used during HITL approvals.

```mermaid
flowchart TD
    TRIAGE[triage_router] -->|cancel actions| DISPATCH[apply_reminder_actions_node]
    TRIAGE -->|respond (create reminder)| DISPATCH
    TRIAGE -->|notify (pending reminder)| HITL[triage_interrupt_handler]
    DISPATCH --> RESPONSE[response_agent]
    DISPATCH --> HITL
    HITL -->|user respond| DISPATCH
    HITL -->|ignore/accept| END[(End)]
    RESPONSE --> END
```

In [ ]:
from datetime import datetime, timedelta, timezone
from pprint import pprint

from email_assistant.graph.reminder_nodes import register_reminder_actions
from email_assistant.tools.reminders import SqliteReminderStore

store = SqliteReminderStore(db_path=":memory:")
store.setup()

batch = register_reminder_actions(
    thread_id="thread-demo",
    actions=[
        {"action": "cancel", "thread_id": "thread-demo"},
        {
            "action": "create",
            "thread_id": "thread-demo",
            "subject": "Follow up invoice",
            "due_at": (datetime.now(timezone.utc) + timedelta(hours=24)).isoformat(),
            "reason": "demo batch",
        },
    ],
    next_node="response_agent",
)

print("Staged actions:")
pprint(batch)

result = store.apply_actions(batch["reminder_actions"])
print("\nDispatcher result:")
pprint(result)

print("\nActive reminders:")
for reminder in store.iter_active_reminders():
    pprint(
        {
            "id": reminder.id,
            "thread_id": reminder.thread_id,
            "subject": reminder.subject,
            "due_at": reminder.due_at.isoformat(),
        }
    )
